In [1]:
import righor
import matplotlib.pyplot as plt
import seaborn
import pandas as pd
from tqdm.notebook import tqdm
from collections import Counter
import numpy as np

# load the model
igor_model = righor.load_model("human", "tra")

# alternatively, you can load a model from igor files 
# igor_model = righor.load_model_from_files(params.txt, marginals.txt, anchor_v.csv, anchor_j.csv)

In [2]:
## Generate sequences

# Create a generator object
generator = igor_model.generator(seed=42) # or igor_model.generator() to run it without a seed

# Generate 10'000 functional sequences (not out-of-frame, no stop codons, right boundaries)
for _ in tqdm(range(10000)):
    # generate_without_errors ignore Igor error model, use "generate" if this is needed
    sequence = generator.generate_without_errors(functional=True)
    if "IGH" in sequence.cdr3_aa:
        print("TRB CDR3 containing \"IGH\":", sequence.cdr3_aa)

print()

# Generate one sequence with a particular V/J genes family 
V_genes = righor.genes_matching("TRAV5", igor_model) # all the V genes that match TRBV5
J_genes = righor.genes_matching("TRAJ", igor_model) # all the J genes
generator = igor_model.generator(seed=42, available_v=V_genes, available_j=J_genes) 
generation_result = generator.generate_without_errors(functional=True)
print("Result:")
print(generation_result)
print("Explicit recombination event:")
print(generation_result.recombination_event)

  0%|          | 0/10000 [00:00<?, ?it/s]

TRB CDR3 containing "IGH": CALIGHSIKAAGNKLTF

Result:
GenerationResult(
CDR3 (nucletides): TGTGCAGAGTTCGCCACCACCAATGCAGGCAAATCAACCTTT,
CDR3 (amino-acids): CAEFATTNAGKSTF,
Full sequence (nucleotides): GGAGAGGATGTGGAGCAGAGTCTTTTCCTG...,
V gene: TRAV5*01,
J gene: TRAJ27*01)
		 
Explicit recombination event:
StaticEvent(
nb. del. on V3: 8,
nb. del. on J5: 8,
V-J insertions: TTCGCCACC)


In [3]:
## Evaluate a given sequence

my_sequence = "ACCCTCCAGTCTGCCAGGCCCTCACATACCTCTCAGTACCTCTGTGCCAGCAGTGAGGACAGGGACGTCACTGAAGCTTTCTTTGGACAAGGCACC"

# first align the sequence
align_params = righor.AlignmentParameters() # default alignment parameters
aligned_sequence = igor_model.align_sequence(my_sequence, align_params)

# then evaluate it
infer_params = righor.InferenceParameters() # default inference parameters
result_inference = igor_model.evaluate(aligned_sequence, infer_params)

# Most likely scenario
best_event = result_inference.best_event

print(f"Probability that this specific event chain created the sequence: {best_event.likelihood / result_inference.likelihood:.2f}.")
print(f"Reconstructed sequence (without errors):", best_event.reconstructed_sequence)
print(f"Pgen: {result_inference.pgen:.1e}")

Probability that this specific event chain created the sequence: 0.59.
Reconstructed sequence (without errors): AGTCAACAGGGAGAAGAGGATCCTCAGGCCTTGAGCATCCAGGAGGGTGAAAATGCCACCATGAACTGCAGTTACAAAACTAGTATAAACAATTTACAGTGGTATAGACAAAATTCAGGTAGAGGCCTTGTCCACCTAATTTTAATACGTTCAAATGAAAGAGAGAAACACAGTGGAAGATTAAGAGTCACGCTTGACACTTCCAAGAAAAGCAGTTCCTTGTTGATCACGGCTTCCCGGGCAGCAGACACTGCTTCTTACTTCTGTGCCAGCAGTGAGGACAGGGACGTCACTAAATACATCTTTGGAACAGGCACCAGGCTGAAGGTTTTAGCAA
Pgen: 3.1e-27


In [4]:
## infer a complete model 

# here we just generate the sequences needed
generator = igor_model.generator()
sequences = [generator.generate(False).full_seq for _ in range(10000)]

# define parameters for the alignment and the inference
align_params = righor.AlignmentParameters()
infer_params = righor.InferenceParameters()

# generate an uniform model as a starting point 
# (it's generally faster to start from an already inferred model)
model = igor_model.uniform()


In [5]:
# align multiple sequences at once (takes ~1min)
aligned_sequences = model.align_all_sequences(sequences, align_params)

In [ ]:
# multiple round of expectation-maximization to infer the model (takes ~1min)
models = {}
models[0] = model
for ii in tqdm(range(10)):
    models[ii+1] = models[ii].copy()
    models[ii+1].infer(aligned_sequences, infer_params)


In [ ]:
fig, ax = plt.subplots()
ax.scatter(range(igor_model.p_del_j_given_j.shape[0]), models[9].p_del_j_given_j.sum(axis=1))
ax.scatter(range(igor_model.p_del_j_given_j.shape[0]), igor_model.p_del_j_given_j.sum(axis=1))

In [ ]:
fig, ax = plt.subplots()
ax.scatter(range(igor_model.p_ins_vj.shape[0]), models[9].p_ins_vj)
ax.scatter(range(igor_model.p_ins_vj.shape[0]), igor_model.p_ins_vj)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(range(igor_model.p_del_v_given_v.shape[0]), models[9].p_del_v_given_v.sum(axis=1))
ax.scatter(range(igor_model.p_del_v_given_v.shape[0]), igor_model.p_del_v_given_v.sum(axis=1))

In [ ]:
models[9].error_rate, igor_model.error_rate